# Data Importing

This notebook will mainly cover the steps I've taken to import and clean the datasets from Kaggle. For more detailed descriptions on the problem statement and methodology of my Capstone Project, please refer to the Part 2 Jupyter Notebook in this Repository.

In [1]:
#Import Libraries

import pandas as pd
import numpy as np
import matplotlib as plt

import json
import requests
import time
import nltk
import regex as re
from collections import Counter
from bs4 import BeautifulSoup

Sources:
- "Convert Yelp Dataset to CSV" by [*George Hou*](https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88)
- General Assembly DSIR 222 Coursework: Hank Butler, John Hazard, Heather Robbins

From the Kaggle Dataset, the 3 main json files that I will be importing and analyzing for my Capstone Project are the business, review and user files.

### Loading and cleaning the business.json file

In [9]:
business_json_path = '../Data/business.json'

In [10]:
df_b = pd.read_json(business_json_path, lines=True)

In [11]:
# 1 = open, 0 = closed
df_b = df_b[df_b['is_open']==1]

In [12]:
# must keep business_id to merge with reviews
drop_columns = ['hours','is_open','review_count']
df_b = df_b.drop(drop_columns, axis=1)

For this Capstone Project, we are only focusing on Restaurant and F&B Businesses. As such, we will be restricting the categories that businesses can contain in our filtered dataset. Based on the above count plot, we will be restricting these broad categories to: Restaurants, Fast Food, Breakfast & Brunch, Cafes.

In [13]:
# split businesses by category
business_restaurant = df_b[df_b['categories'].str.contains(
              'Restaurant',
              case=False, na=False)]

In [14]:
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

In [15]:
df_explode.categories.value_counts()

Restaurants      42237
Shopping         26734
Food             23208
Home Services    18455
Beauty & Spas    16545
                 ...  
Geneticists          1
Makerspaces          1
Fencing Clubs        1
Flyboarding          1
Churros              1
Name: categories, Length: 1290, dtype: int64

In [16]:
df_explode[df_explode.categories.str.contains('Restaurant',
                      case=True,na=False)].categories.value_counts()

Restaurants            42237
Restaurant Supplies       20
Pop-Up Restaurants        10
Name: categories, dtype: int64

In [17]:
business_restaurant.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese"
11,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco..."
13,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings"
23,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,"Center Core - Food Court, Fl 3, Pittsburgh Int...",Pittsburgh,PA,15231,40.496177,-80.246011,4.0,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Sandwiches, Salad, Restaurants, Burgers, Comfo..."


In [18]:
business_restaurant.shape

(42252, 11)

In [19]:
business_restaurant['state'].value_counts()

ON     10507
AZ      7680
NV      5124
QC      4397
OH      4209
NC      3174
PA      2933
AB      2315
WI      1167
IL       457
SC       264
NY        12
XWY        2
CON        1
AR         1
XGM        1
VT         1
BC         1
NM         1
WA         1
CA         1
NE         1
VA         1
AL         1
Name: state, dtype: int64

In [20]:
csv_bus = "../Data/businesses.csv"
business_restaurant.to_csv(csv_bus, index=False)

### Loading and cleaning the review.json file

In [21]:
review_json_path = '../Data/review.json'

In [22]:
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                     chunksize=size)

In [24]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_restaurant, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)

542224 out of 1,000,000 related reviews
528775 out of 1,000,000 related reviews
536867 out of 1,000,000 related reviews
514039 out of 1,000,000 related reviews
543781 out of 1,000,000 related reviews
538411 out of 1,000,000 related reviews
362868 out of 1,000,000 related reviews


In [30]:
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

### Converting the dataframe to a new CSV file

In [31]:
csv_name = "yelp_restaurant_reviews.csv"
df.to_csv(csv_name, index=False)